In [1]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<a target="_blank" href="https://colab.research.google.com/github/NVIDIA-Merlin/dataloader/blob/main/examples/01a-Getting-started-Tensorflow.ipynb" style="float:left">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# Getting Started with Merlin dataloader and TensorFlow

This notebook is created using the latest stable [merlin-tensorflow](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-tensorflow) container.

## Overview

[Merlin dataloader](https://github.com/NVIDIA-Merlin/dataloader) is a library for constructing highly optimized dataloaders to accelerate training pipelines in TensorFlow (Keras) and PyTorch. In this example, we will provide a simple pipeline to train a MatrixFactorization Model in TensorFlow with Merlin dataloader based on the MovieLens dataset.

The core features of Merlin dataloader:

- Accelerate pipelines by up to 10x compared to other dataloaders
- Handles larger than memory dataset by streaming data from disk
- Support for common data formats: CSV, Parquet, Avro
- Distributed training support

### Learning objectives

- Using Merlin dataloader to train a TensorFlow Keras Model

# Downloading and preparing the dataset

We will base our example on the  [MovieLens25M](https://grouplens.org/datasets/movielens/25m/) dataset.

In [2]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    !pip install merlin-core  #noqa
    !pip install merlin-dataloader

from merlin.core.utils import download_file
from merlin.core.dispatch import get_lib

from merlin.io import Dataset
from merlin.loader.tensorflow import Loader

import tensorflow as tf

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-01-04 02:53:10.816796: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-04 02:53:10.817325: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-04 02:53:10.817466: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
DATA_PATH = '/workspace'

In [4]:
download_file("http://files.grouplens.org/datasets/movielens/ml-25m.zip", DATA_PATH + "/ml-25m.zip")

downloading ml-25m.zip: 262MB [01:15, 3.48MB/s]                                                                         
unzipping files: 100%|█████████████████████████████████████████████████████████████████| 8/8 [00:04<00:00,  1.99files/s]


# Training a TensorFlow Keras Model with Merlin dataloader

In [5]:
ratings = get_lib().read_csv(DATA_PATH + '/ml-25m/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


The `ratings.csv` file stores ratings a user has given a movie. Let's load the data directly from disk into a `Merlin Dataset` and train a simple `MatrixFactorization` model that we will construct in `Tensorflow`.

In [6]:
dataset = Dataset(DATA_PATH + '/ml-25m/ratings.csv')

Let us now instantiate the `dataloader`.

In [7]:
loader = Loader(dataset, batch_size=65536)

As is, the `loader` will output a batch that will consist of a tuple with dictionary with tensors and `None`.

In [8]:
loader.peek()

2023-01-04 02:54:34.188473: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-04 02:54:34.190430: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-04 02:54:34.190611: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-04 02:54:34.190748: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning

({'userId': <tf.Tensor: shape=(65536, 1), dtype=int64, numpy=
  array([[ 87932],
         [120961],
         [ 97155],
         ...,
         [ 63329],
         [  8705],
         [ 54183]])>,
  'movieId': <tf.Tensor: shape=(65536, 1), dtype=int64, numpy=
  array([[  377],
         [82378],
         [45672],
         ...,
         [   34],
         [ 5618],
         [ 6664]])>,
  'timestamp': <tf.Tensor: shape=(65536, 1), dtype=int64, numpy=
  array([[ 976158320],
         [1459110489],
         [1523551383],
         ...,
         [ 837197578],
         [1268121690],
         [1355024867]])>,
  'rating': <tf.Tensor: shape=(65536, 1), dtype=float64, numpy=
  array([[5. ],
         [3. ],
         [4. ],
         ...,
         [4. ],
         [4.5],
         [3. ]])>},
 None)

TensorFlow Kera's .fit function expects to receive the data as a tuple (x, y), with x being the input features and y the label. We need to provide this information to the dataloader. We can add a custom function to convert the data into the tuple with `process_batch`.

In [9]:
label_column = 'rating'


def process_batch(data, _):
    x = {col: data[col] for col in data.keys() if col != label_column}
    y = data[label_column]
    return (x, y)


loader = loader.map(process_batch)
loader.stop()

We now have the data in the shape that `Tensorflow` expects.

In [10]:
loader.peek()

({'userId': <tf.Tensor: shape=(65536, 1), dtype=int64, numpy=
  array([[ 31844],
         [ 60794],
         [145909],
         ...,
         [ 59919],
         [  5462],
         [ 80446]])>,
  'movieId': <tf.Tensor: shape=(65536, 1), dtype=int64, numpy=
  array([[  1210],
         [106489],
         [  1953],
         ...,
         [  5502],
         [   832],
         [  2442]])>,
  'timestamp': <tf.Tensor: shape=(65536, 1), dtype=int64, numpy=
  array([[1383324340],
         [1485347965],
         [ 968341104],
         ...,
         [1439803083],
         [1103231279],
         [1314837653]])>},
 <tf.Tensor: shape=(65536, 1), dtype=float64, numpy=
 array([[3.5],
        [3. ],
        [2. ],
        ...,
        [2.5],
        [3. ],
        [3.5]])>)

Let us now construct a simple MatrixFactorization model and train for a single epoch.

In [11]:
class MatrixFactorization(tf.keras.Model):
    def __init__(self, n_factors):
        super().__init__()
        self.user_embeddings = tf.keras.layers.Embedding(ratings['userId'].max() + 1, n_factors)
        self.movie_embeddings = tf.keras.layers.Embedding(ratings['movieId'].max() + 1, n_factors)

    def call(self, batch, training=False):
        user_embs = self.user_embeddings(batch['userId'])
        movie_embs = self.movie_embeddings(batch['movieId'])

        tensor = (tf.squeeze(user_embs) * tf.squeeze(movie_embs))
        return tf.reduce_sum(tensor, 1)

In [12]:
model = MatrixFactorization(64)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-2), loss=tf.keras.losses.MeanSquaredError())

Let us now train for a single epoch.

In [13]:
model.fit(loader, epochs=1)

2023-01-04 02:54:37.125481: W tensorflow/core/common_runtime/forward_type_inference.cc:231] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_BOOL
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_LEGACY_VARIANT
    }
  }
}

	while inferring type of node 'mean_squared_error/cond/output/_11'


382/382 [==============================] - 4s 8ms/step - loss: 2.1479


In [14]:
model.evaluate(loader)

382/382 [==============================] - 1s 2ms/step - loss: 0.6763


0.6762918829917908

## Conclusion

We demonstrated how to train a TensorFlow Keras model with Merlin dataloader. Merlin dataloader can accelerate existing TensorFlow pipelines with minimal code changes. 

# Next Steps

Merlin dataloader is part of NVIDIA Merlin, a open source framework for recommender systems. In this example, we looked only on a specific use-case to accelerate existing training pipelines. We provide more libraries to make recommender system pipelines easier:

* [NVTabular](https://github.com/NVIDIA-Merlin/NVTabular) is a library to accelerate and scale feature engineering
* [Merlin Models](https://github.com/NVIDIA-Merlin/models) is a library with high-quality implementations of popular recommender systems architectures

The libraries are designed to work closely together. We recommend to check out our examples:

* [Getting Started with NVTabular: Process Tabular Data On GPU](https://github.com/NVIDIA-Merlin/NVTabular/blob/main/examples/01-Getting-started.ipynb)
* [Getting Started with Merlin Models: Develop a Model for MovieLens](https://github.com/NVIDIA-Merlin/models/blob/main/examples/01-Getting-started.ipynb)

In the example, [From ETL to Training RecSys models - NVTabular and Merlin Models integrated example](https://github.com/NVIDIA-Merlin/models/blob/main/examples/02-Merlin-Models-and-NVTabular-integration.ipynb), we explain how the close collaboration works.